In [ ]:
!pip install transformers

  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [5]:
%cd /gdrive/MyDrive/sentiment_analysis_project/
%ls

/gdrive/MyDrive/sentiment_analysis_project
bert-tokenizer/  test_15K.pkl  train_15K.pkl  trainer_sentiment_analysis.ipynb


In [14]:
import pickle

import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer


def load_pickle_datset(path):
    return pickle.load(open(path, 'rb'))


class MovieDataset(Dataset):
    def __init__(self, path):
        self.tokenizer = BertTokenizer.from_pretrained('./bert-tokenizer')
        MAX_LEN = 512
        data = load_pickle_datset(path)
        self.x = data['review'].tolist()
        self.input_lengths = []
        for i, v in enumerate(self.x):
            truncated = self.tokenizer.tokenize(v)[:MAX_LEN - 2]
            self.input_lengths.append(len(truncated) + 2)
            v = ['[CLS]'] + truncated + ['[SEP]']
            v = v + ['[PAD]'] * (MAX_LEN - len(v))
            self.x[i] = self.tokenizer.convert_tokens_to_ids(v)

        self.x = torch.tensor(self.x)
        self.y = torch.tensor(data['sentiment'].values)
        self.input_lengths = torch.tensor(self.input_lengths)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        return self.x[idx], self.y[idx], self.input_lengths[idx]


In [15]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence


class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, lstm_out, lstm_layers, out_dim, dropout_lstm, dropout):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.bilstm = nn.LSTM(input_size=embed_size, hidden_size=lstm_out, num_layers=lstm_layers, batch_first=True,
                              dropout=dropout_lstm, bidirectional=True)
        self.dense1 = nn.Linear(lstm_out * 2, 20)
        self.dense2 = nn.Linear(20, 10)
        self.output = nn.Linear(10, out_dim)
        self.leaky_relu = nn.LeakyReLU()
        self.sigmoid = nn.Sigmoid()

        self.drop1 = nn.Dropout(dropout)
        self.drop2 = nn.Dropout(dropout)

    def forward(self, input_ids, input_lengths, targets=None):
        x = self.embedding(input_ids)
        x = self.drop1(x)
        packed_x = pack_padded_sequence(x, input_lengths, batch_first=True, enforce_sorted=False)
        output, (h_n, c_n) = self.bilstm(packed_x)
        output_unpacked, output_lengths = pad_packed_sequence(output, batch_first=True)
        out = output_unpacked[:, 0, :]
        out = self.leaky_relu(out)

        out = self.leaky_relu(self.drop2(self.dense1(out)))
        out = self.leaky_relu(self.drop2(self.dense2(out)))

        out = self.output(out)
        out = self.sigmoid(out)

        out = out.view(-1)
        if targets is not None:
            criterion = nn.BCELoss()
            loss = criterion(out, targets.float())
            return out, loss
        else:
            return out,


In [18]:
import time

import torch
from sklearn.metrics import classification_report
from tqdm import tqdm
import warnings

# from data_loaders import MovieDataset
# from logger import get_logger
# from models import LSTMModel


def train_step(model, batch, optimizer):
    model.train()
    optimizer.zero_grad()
    input_ids, targets, input_lengths = batch
    out, loss = model(input_ids=input_ids.to(device),
                      input_lengths=input_lengths.tolist(),
                      targets=targets.to(device))
    loss.backward()
    optimizer.step()
    return loss.item()


def eval_step(model, batch):
    model.eval()
    input_ids, targets, input_lengths = batch
    out, loss = model(input_ids=input_ids.to(device),
                      input_lengths=input_lengths.tolist(),
                      targets=targets.to(device))

    return loss.item()


def predicting(model, batch):
    model.eval()
    input_ids, _, input_lengths = batch
    out, = model(input_ids=input_ids.to(device),
                 input_lengths=input_lengths.tolist())
    return out


def get_class(t):
    return (t >= 0.5) * 1


def save_model(model, epoch, optimizer, loss, path):
    torch.save({'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss}, path)


if __name__ == '__main__':
    warnings.filterwarnings("ignore")

    # logger = get_logger('trainer.log')

    # logger.info('Loading the train data')
    print('Loading the train data')
    train_dataset = MovieDataset('train_15K.pkl')
    train_dataloader = torch.utils.data.DataLoader(train_dataset,
                                                   batch_size=1000, shuffle=True,
                                                   num_workers=1)
    # logger.info('Finished loading validation data')
    print('Finished loading validation data')

    # logger.info('Loading the evaluation data')
    print('Loading the evaluation data')
    eval_dataset = MovieDataset('test_15K.pkl')
    eval_dataloader = torch.utils.data.DataLoader(eval_dataset,
                                                  batch_size=1000, shuffle=True,
                                                  num_workers=1)
    # logger.info('Finished loading evaluation data')
    print('Finished loading evaluation data')

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # logger.info('Initializing model')
    print('Initializing model')
    model = LSTMModel(vocab_size=train_dataset.tokenizer.vocab_size,
                      embed_size=50,
                      lstm_out=50,
                      lstm_layers=2,
                      out_dim=1,
                      dropout_lstm=0.6,
                      dropout=0.6)
    # logger.info('Finished initializing model')
    print('Finished initializing model')

    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

    eval_steps = 5
    loggging_steps = 1
    reporting_steps = 5

    min_eval_loss = float('inf')
    break_point = 0
    if_not_better_in = 5

    # logger.info('Starting epochs ...')
    print('Starting epochs ...')
    for epoch in range(1000):

        running_train_loss = 0.0
        for batch in tqdm(train_dataloader, total=len(train_dataloader)):
            running_train_loss += train_step(model, batch, optimizer)

        if (epoch + 1) % loggging_steps == 0:
            time.sleep(0.5)
            # logger.info('Epoch: %s || Train Loss: %s' % (epoch + 1, running_train_loss))
            print('Epoch: %s || Train Loss: %s' % (epoch + 1, running_train_loss))
            time.sleep(0.5)

        if (epoch + 1) % eval_steps == 0:
            with torch.no_grad():
                running_eval_loss = 0.0
                for batch in tqdm(eval_dataloader, total=len(eval_dataloader)):
                    running_eval_loss += eval_step(model, batch)
                time.sleep(0.5)
                # logger.info('Epoch: %s || Validation Loss: %s' % (epoch + 1, running_eval_loss))
                print('Epoch: %s || Validation Loss: %s' % (epoch + 1, running_eval_loss))
                time.sleep(0.5)

                if running_eval_loss < min_eval_loss:
                    min_eval_loss = running_eval_loss
                    save_model(model=model, epoch=epoch+1, optimizer=optimizer, 
                               loss=min_eval_loss, path=f'models/model-{epoch+1}.pt')
                    break_point = 0
                else:
                    break_point += 1
                
                if break_point >= if_not_better_in:
                    print(f'Stopping iteration because model did not get better in the last {break_point * eval_steps} epochs')
                    break

        if (epoch + 1) % reporting_steps == 0:
            y_preds = []
            y_trues = []
            with torch.no_grad():
                for batch in tqdm(eval_dataloader, total=len(eval_dataloader)):
                    _, y_true, _ = batch
                    y_pred = predicting(model, batch)
                    y_pred = get_class(y_pred)
                    
                    y_preds.extend(y_pred.view(-1).tolist())
                    y_trues.extend(y_true.view(-1).tolist())

                r = classification_report(y_trues, y_preds)
                time.sleep(0.5)
                # logger.info(f'\n{r}\n')
                print(f'\n{r}\n')
                time.sleep(0.5)


Loading the train data
Finished loading validation data
Loading the evaluation data
Finished loading evaluation data
Initializing model
Finished initializing model
Starting epochs ...


100%|██████████| 12/12 [00:35<00:00,  2.93s/it]


Epoch: 1 || Train Loss: 8.323504030704498


100%|██████████| 12/12 [00:35<00:00,  2.93s/it]


Epoch: 2 || Train Loss: 8.31876277923584


...

Epoch: 79 || Train Loss: 2.174392595887184


100%|██████████| 12/12 [00:35<00:00,  2.94s/it]


Epoch: 80 || Train Loss: 1.799319252371788


100%|██████████| 3/3 [00:01<00:00,  2.86it/s]


Epoch: 80 || Validation Loss: 1.769592821598053
Stopping iteration because model did not get better in the last 25 epochs
